Notes: 
- FSQR ratings need to be halved to standardize with yelp

In [79]:
import pandas as pd
import sqlite3

Join the data from Part 1 with the data from Part 2 to create a new dataframe.

In [80]:
# load part 2 data
results_FSQR = pd.read_csv('../data/POI_results/pois_FSQR.csv')
results_YELP = pd.read_csv('../data/POI_results/pois_YELP.csv')

In [81]:
# validate load
print(results_FSQR.info())
results_YELP.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 370 entries, 0 to 369
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   station_name  370 non-null    object 
 1   station_id    370 non-null    object 
 2   API           370 non-null    object 
 3   id            370 non-null    object 
 4   category      370 non-null    object 
 5   distance      370 non-null    int64  
 6   name          370 non-null    object 
 7   rating        219 non-null    float64
 8   ratings       219 non-null    float64
dtypes: float64(2), int64(1), object(6)
memory usage: 26.1+ KB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 370 entries, 0 to 369
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   station_name  370 non-null    object 
 1   station_id    370 non-null    object 
 2   API           370 non-null    object 
 3   id            370 non-null    o

In [82]:
# join part 2 data
palmaQuery = pd.concat([results_FSQR, results_YELP], ignore_index = True)

In [83]:
# validate join
palmaQuery.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 740 entries, 0 to 739
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   station_name  740 non-null    object 
 1   station_id    740 non-null    object 
 2   API           740 non-null    object 
 3   id            740 non-null    object 
 4   category      740 non-null    object 
 5   distance      740 non-null    float64
 6   name          740 non-null    object 
 7   rating        589 non-null    float64
 8   ratings       589 non-null    float64
dtypes: float64(3), object(6)
memory usage: 52.2+ KB


Provide a visualization that you used as part of your EDA process. Explain the initial pattern or relationship you discoved through this visualization. 

# Database

Put all your results in an SQLite3 database (remember, SQLite stores its databases as files in your local machine - make sure to create your database in your project's data/ directory!)

In [84]:
# table creation queries
createTable_networks = '''CREATE TABLE networks
(
    network_id  VARCHAR PRIMARY KEY,
    company     VARCHAR,
    name        VARCHAR,
    city        VARCHAR,
    country     VARCHAR,
    latitude    FLOAT,
    longitude   FLOAT
)'''
createTable_stations = '''CREATE TABLE stations
(
    station_id  VARCHAR PRIMARY KEY,
    network_id  VARCHAR,
    name        VARCHAR,
    bikes       INTEGER,
    latitude    FLOAT,
    longitude   FLOAT
)'''
createTable_palmaQuery = '''CREATE TABLE palmaQuery
(
    station_id    VARCHAR,
    station_name  VARCHAR,
    API           VARCHAR,
    id            VARCHAR,
    category      VARCHAR,
    distance      FLOAT,
    name          VARCHAR,
    rating        FLOAT,
    ratings       FLOAT
)'''

In [85]:
# open database connection
conn_str = "../data/Biketouring.db"

conn = sqlite3.connect(conn_str) 
c = conn.cursor()
print('connection established')

connection established


In [86]:
# drop tables with duplicate name
c.execute("DROP TABLE IF EXISTS networks;")
c.execute("DROP TABLE IF EXISTS stations;")
c.execute("DROP TABLE IF EXISTS palmaQuery;")

In [87]:
# create tables
c.execute(createTable_networks)
c.execute(createTable_stations)
c.execute(createTable_palmaQuery)

print('tables created')

tables created


In [88]:
networks = pd.read_csv('../data/networks_all.csv')
networks.to_sql('networks', conn, if_exists='replace', index=False)

stations = pd.read_csv('../data/Networks/palmaBikes.csv')
stations.to_sql('stations', conn, if_exists='replace', index=False)

palmaQuery.to_sql('palmaQuery', conn, if_exists='replace', index=False)

740

In [93]:
sql = 'SELECT * FROM networks'
pd.read_sql_query(sql, conn)

,company,network_id,name,city,country,latitude,longitude
0,['ЗАО «СитиБайк»'],velobike-moscow,Velobike,Moscow,RU,55.750000,37.616667
1,['Urban Infrastructure Partner'],baerum-bysykkel,Bysykkel,Bærum,NO,59.894550,10.546343
2,['Comunicare S.r.l.'],bicincitta-siena,Bicincittà,Siena,IT,43.318600,11.330600
3,['Cyclopolis Systems'],cyclopolis-maroussi,Cyclopolis,Maroussi,GR,38.056872,23.808330
4,"['Groundwork', 'Slough Borough Council', 'ITS']",cycle-hire-slough,Cycle Hire,Slough,GB,51.511350,-0.591562
...,...,...,...,...,...,...,...
610,None,acces-velo-saguenay,Accès Vélo,Saguenay,CA,48.433333,-71.083333
611,"['Vilvolt CA Épinal', ""Communauté d'agglomérat...",vilvolt,Vilvolt,Epinal,FR,48.183300,6.450800
612,"['VeloBaie', 'Fifteen SAS']",velobaie,VeloBaie,Saint-Brieuc,FR,48.514000,-2.765000
613,['Donkey Republic'],donkey-gh,Donkey Republic - Gent,Gent,BE,51.050000,3.730300


In [94]:
# commit and close
conn.commit()
conn.close()
print('import to database completed')

import to database completed


Look at the data before and after the join to validate your data.